# VK friends graph

In [60]:
import vk
import pandas as pd
import numpy as np
import matplotlib
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
my_id = 91857120

In [61]:
session = vk.Session()
api = vk.API(session)
api.users.get(user_ids=1)

[{'first_name': 'Павел', 'last_name': 'Дуров', 'uid': 1}]

In [62]:
friends = pd.DataFrame(api.friends.get(user_id = my_id, fields = 'sex, bdate, city, country, timezone, \
photo_200_orig, has_mobile, education, relation, status, universities'))

In [63]:
friends.isnull().sum()

bdate                22
city                  5
country               5
deactivated         100
education_form       89
education_status     88
faculty              77
faculty_name         77
first_name            0
graduation           77
has_mobile            5
hidden               59
last_name             0
lists                76
photo_200_orig        0
relation             77
relation_partner    102
sex                   0
status                5
status_audio        103
uid                   0
universities         77
university           77
university_name      77
user_id               0
dtype: int64

In [64]:
friends.drop(['relation_partner', 'uid', 'university', 'lists', 'hidden', 'has_mobile'], axis=1, inplace=True)

In [65]:
friends.drop(['relation'], axis=1, inplace=True)

In [66]:
friends.isnull().sum()/len(friends)

bdate               0.209524
city                0.047619
country             0.047619
deactivated         0.952381
education_form      0.847619
education_status    0.838095
faculty             0.733333
faculty_name        0.733333
first_name          0.000000
graduation          0.733333
last_name           0.000000
photo_200_orig      0.000000
sex                 0.000000
status              0.047619
status_audio        0.980952
universities        0.733333
university_name     0.733333
user_id             0.000000
dtype: float64

In [76]:
friends.head()

,bdate,city,country,deactivated,education_form,education_status,faculty,faculty_name,first_name,graduation,last_name,photo_200_orig,sex,status,status_audio,universities,university_name,user_id
0,14.4.1988,1,1,NaN,NaN,NaN,NaN,NaN,Ольга,NaN,Ильина,http://cs419226.vk.me/v419226076/b358/-uTV5JdE...,1,,NaN,NaN,NaN,173076
1,NaN,1,1,NaN,NaN,NaN,NaN,NaN,Александр,NaN,Семёнов,http://cs61.vk.me/u180827/a_41350d36.jpg,2,http://semenoffalex.ru,NaN,NaN,NaN,180827
2,21.9.1993,2,1,NaN,NaN,NaN,NaN,NaN,Иван,NaN,Мороз,http://cs627621.vk.me/v627621072/e642/OexyWrjS...,2,,NaN,NaN,NaN,452072
3,21.12,1,1,NaN,NaN,NaN,0,,Роман,2009,Вейнберг,http://cs627822.vk.me/v627822588/37aef/wFcABY_...,2,В жизни нельзя купить три настоящие вещи: любо...,NaN,"[{'name': 'РЭУ им. Г. В. Плеханова', 'id': 326...",РЭУ им. Г. В. Плеханова,546588
4,NaN,151,1,NaN,NaN,NaN,NaN,NaN,Ирина,NaN,Носова,http://cs9625.vk.me/u693176/a_06e48837.jpg,1,,NaN,NaN,NaN,693176
